In [1]:
# -*- coding: utf-8 -*-

import logging
from colorama import Fore
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display
import ecomis
import xlwings as xw
import pandas as pd
import numpy as np
import posixpath
import dateutil.relativedelta

logger = logging.getLogger("juplogger")
handler = ecomis.LogViewver()
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.propagate = False

month = '202008'

inifile = ecomis.ReadIniFile(mercado="empresas")

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = ecomis.DbDataProcess(month)
dbmanager.setParser(dbparser)# Configurando los archivos de configuración para administrar la base de datos

handler.show_logs()


Output(layout=Layout(border='solid', height='600px', width='100%'), outputs=({'name': 'stdout', 'output_type':…

In [3]:
section = ecomis.SectionObj(inifile,"ResultadosComisiones_All_Files",month)
loader = ecomis.LoadFileProcess(section)
df = loader.loadFile()
delta_mes = dateutil.relativedelta.relativedelta(months=1)
df['FECHA_DE_PRODUCCION'] = df['FECHA_DE_PAGO'].apply(lambda x: x - delta_mes)
df = df[df["ESQUEMA"] != "SPARE POSITION"]



In [4]:
dbmanager.dbOperation('insert','tblResultadosComisiones',df)

In [14]:
indicadores_path = inifile.getMainPath() + '/Indicadores'
writer = pd.ExcelWriter(posixpath.join(indicadores_path, month + "_Indicadores.xlsx"), engine='xlsxwriter')
df.to_excel(writer, sheet_name = 'Indicadores')
writer.save()

In [17]:
df.dtypes

DNI                                   object
GERENCIA1                             object
GERENCIA2                             object
ZONA                                  object
ESQUEMA                               object
PORCENTAJE_TOTAL_PONDERADO           float64
CAPTURA_I                            float64
GESTION_I                            float64
DESARROLLO_I                         float64
TARGET                                 int64
PAGO                                 float64
FECHA_DE_PAGO                 datetime64[ns]
COMISIÓN                             float64
FECHA_DE_PRODUCCION           datetime64[ns]
dtype: object